In [10]:
from custom_dataset.segdataset import InstanceSegmentationDataset
from transformers import SegformerImageProcessor
from torch.utils.data import DataLoader
import json

In [11]:
# define model
from transformers import SegformerForSemanticSegmentation
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b5",
                                                         num_labels=5)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from models import custom_segformer

In [4]:
model.config.output_hidden_states = True

In [5]:
import torch
dummy_image = torch.rand(1, 3, 512, 512)

In [6]:
out = model(dummy_image)

In [7]:
out.keys()

odict_keys(['logits', 'hidden_states'])

In [8]:
out_hidden_states = out['hidden_states']

In [9]:
out_hidden_states[0].shape,out_hidden_states[1].shape,out_hidden_states[2].shape,out_hidden_states[3].shape

(torch.Size([1, 64, 128, 128]),
 torch.Size([1, 128, 64, 64]),
 torch.Size([1, 320, 32, 32]),
 torch.Size([1, 512, 16, 16]))

In [11]:
from models.custom_segformer import SegformerDecodeHead

In [12]:
model = SegformerDecodeHead.from_pretrained("nvidia/mit-b5",num_labels=5)

Some weights of SegformerDecodeHead were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['segformer.batch_norm.bias', 'segformer.batch_norm.num_batches_tracked', 'segformer.batch_norm.running_mean', 'segformer.batch_norm.running_var', 'segformer.batch_norm.weight', 'segformer.classifier.bias', 'segformer.classifier.weight', 'segformer.linear_c.0.proj.bias', 'segformer.linear_c.0.proj.weight', 'segformer.linear_c.1.proj.bias', 'segformer.linear_c.1.proj.weight', 'segformer.linear_c.2.proj.bias', 'segformer.linear_c.2.proj.weight', 'segformer.linear_c.3.proj.bias', 'segformer.linear_c.3.proj.weight', 'segformer.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
out = model(out_hidden_states)

In [14]:
out.shape

torch.Size([1, 5, 128, 128])

In [16]:
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import os
import evaluate
import logging
from utils import rs_utils

In [21]:
# Set up a logger
log_dir = "/home/eric/srcs/FewShotSeg_Lab/FewShotVision_Lab/Segmentation_Pipes/logs"
os.makedirs(log_dir, exist_ok=True)
log_file_path = os.path.join(log_dir, "2.log")
logger = rs_utils.setup_logger("FewShotSeg", log_file_path, level=logging.INFO)

In [22]:
# define metric
metric = evaluate.load("mean_iou")
# define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006)
# move model to GPU
device = torch.device("cuda:0")
model.to(device)
model.train()

#-----------
_EPOCHS = 50
_MODEL_SAVE = True
#----------

In [23]:
best_val_iou = 0 

#---
for epoch in range(_EPOCHS):  # loop over the dataset multiple times
    logger.info(f"Epoch: {epoch}")

    # Training loop
    for idx, batch in enumerate(tqdm(train_dataloader)):
        # get the inputs
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits

        loss.backward()
        optimizer.step()

        # Evaluate periodically
        if idx % 100 == 0:
            with torch.no_grad():
                upsampled_logits = nn.functional.interpolate(
                    logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
                )
                predicted = upsampled_logits.argmax(dim=1)

                metrics = metric._compute(
                    predictions=predicted.cpu(),
                    references=labels.cpu(),
                    num_labels=len(id2label),
                    ignore_index=255,
                    reduce_labels=False,  # we've already reduced the labels ourselves
                )
                logger.info(f"Epoch: {epoch}")
                logger.info(f"Loss: {loss.item()}")
                logger.info(f"Mean_iou: {metrics['mean_iou']}")
                logger.info(f"Mean accuracy: {metrics['mean_accuracy']}")

    # Validation loop
    val_loss = 0
    val_metrics = {"mean_iou": 0, "mean_accuracy": 0}
    for batch in tqdm(valid_dataloader):
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            outputs = model(pixel_values=pixel_values, labels=labels)
            val_loss += outputs.loss.item()

            logits = outputs.logits
            upsampled_logits = nn.functional.interpolate(
                logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
            )
            predicted = upsampled_logits.argmax(dim=1)

            batch_metrics = metric._compute(
                predictions=predicted.cpu(),
                references=labels.cpu(),
                num_labels=len(id2label),
                ignore_index=255,
                reduce_labels=False,
            )
            val_metrics["mean_iou"] += batch_metrics["mean_iou"]
            val_metrics["mean_accuracy"] += batch_metrics["mean_accuracy"]

    # Average validation loss and metrics
    val_loss /= len(valid_dataloader)
    val_metrics["mean_iou"] /= len(valid_dataloader)
    val_metrics["mean_accuracy"] /= len(valid_dataloader)

    logger.info(f"Validation Loss: {val_loss}")
    logger.info(f"Validation Mean IoU: {val_metrics['mean_iou']}")
    logger.info(f"Validation Mean Accuracy: {val_metrics['mean_accuracy']}")

    # Save the model if the validation IoU improves
    if val_metrics["mean_iou"] > best_val_iou:
        best_val_iou = val_metrics["mean_iou"]
        
        if _MODEL_SAVE:
            torch.save(
                model.state_dict(),
                os.path.join(
                    "/disk3/eric/checkpoints/military_fewshot_seg",
                    f"segformer_best_epoch_{epoch}_miou_{best_val_iou:.4f}.pt",
                ),
            )
        logger.info("Model saved!")


2024-11-24 22:17:08 - FewShotSeg - INFO - Epoch: 0
2024-11-24 22:17:08 - FewShotSeg - INFO - Epoch: 0


  0%|          | 0/33 [00:00<?, ?it/s]

filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/T140Angara__part_23.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/T140Angara__part_23.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M2A1Slammer2__part_9.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M2A1Slammer2__part_9.png


/home/eric/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/home/eric/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
2024-11-24 22:17:08 - FewShotSeg - INFO - Epoch: 0
2024-11-24 22:17:08 - FewShotSeg - INFO - Epoch: 0
2024-11-24 22:17:08 - FewShotSeg - INFO - Loss: 0.0027527695056051016
2024-11-24 22:17:08 - FewShotSeg - INFO - Loss: 0.0027527695056051016
2024-11-24 22:17:08 - FewShotSeg - INFO - Mean_iou: 0.9384517511960176
2024-11-24 22:17:08 - FewShotSeg - INFO - Mean_iou: 0.9384517511960176
2024-11-24 22:17:08 - FewShotSeg - INFO - Mean accuracy: 0.984277184389754

filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/T140Angara__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/T140Angara__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M5Sandstorm__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M5Sandstorm__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M2A1Slammer2__part_17.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M2A1Slammer2__part_17.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M5Sandstorm__part_24.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M5Sandstorm__part_24.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/T140Angara__part_22.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON

...

  0%|          | 0/33 [00:00<?, ?it/s]

filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/T140Angara__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/T140Angara__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/T140Angara__part_22.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/T140Angara__part_22.png


2024-11-24 22:22:23 - FewShotSeg - INFO - Epoch: 34
2024-11-24 22:22:23 - FewShotSeg - INFO - Epoch: 34
2024-11-24 22:22:23 - FewShotSeg - INFO - Loss: 0.001061049522832036
2024-11-24 22:22:23 - FewShotSeg - INFO - Loss: 0.001061049522832036
2024-11-24 22:22:23 - FewShotSeg - INFO - Mean_iou: 0.9801263076276059
2024-11-24 22:22:23 - FewShotSeg - INFO - Mean_iou: 0.9801263076276059
2024-11-24 22:22:23 - FewShotSeg - INFO - Mean accuracy: 0.9839965589930866
2024-11-24 22:22:23 - FewShotSeg - INFO - Mean accuracy: 0.9839965589930866


filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/T140Angara__part_20.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/T140Angara__part_20.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/Zamak__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/Zamak__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M2A1Slammer2__part_15.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M2A1Slammer2__part_15.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M2A1Slammer2__part_19.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M2A1Slammer2__part_19.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M2A1Slammer2__part_11.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segme

KeyboardInterrupt: 